# [Aligning Big Brains and Atlases](https://biop.github.io/ijp-imagetoatlas/) in Python

This series of notebook demoes the use of ABBA with python.

If you managed to create an environment with PyImageJ and DeepSlice, you will be able, by running this notebook, to perform an automated registration of a qupath project containind mouse brain sections to the Adult Mouse Allen Brain atlas.

Take care : in this notebook, the slices should be ordered, not flipped, orientated correctly, and you need to adjust the display settings for deepslice to work. Also, it's assumed that channel 0 is DAPI and channel 1 is autofluorescence.

You may download and create a project out of this dataset: https://zenodo.org/record/6553641#.Yz6EHkxBxD8. Remove labels and overview in QuPath before running this notebook

It's probably best not to run it headless to check the project before running the registration


Multichannel registration works reasonably well in this notebook because the first channel of the atlas (NISSL, indexed 0), is ressembling the DAPI channel of the slices (indexed 0), and the second channel of the atlas (ARA, indexed 1), is ressembling the autofluorescence channel of the slices (indexed 1).

NOTE : You need to set the elastix and transformix path for the elastix registration steps.

In [1]:
# core dependencies
import os
import time
from pathlib import Path

from bg_atlasapi import show_atlases
from bg_atlasapi import utils

from abba_python.abba import Abba

## 1. ABBA initialization

In [2]:
headless = False

# -- FOR DEBUGGING
# import imagej.doctor
# imagej.doctor.checkup()
# imagej.doctor.debug_to_stderr()

if headless:
    # -- HEADLESS
    aligner = Abba('Adult Mouse Brain - Allen Brain Atlas V3p1', headless=True)
else:
    # -- NOT HEADLESS
    aligner = Abba('Adult Mouse Brain - Allen Brain Atlas V3p1')
    aligner.show_bdv_ui()  # creates and show a bdv view

[java.lang.Enum.toString] Spimdata mpicbg.spim.data.SpimData@3bea0692 is using the Bdv Playground global cache.
[java.lang.Enum.toString] [WARNING] Ignoring unsupported output: ba [ch.epfl.biop.atlas.mouse.allen.ccfv3p1.command.AllenBrainAdultMouseAtlasCCF2017v3p1Command]
[java.lang.Enum.toString] Creating MultiSlicePositioner instance
[java.lang.Enum.toString] MultiSlicePositioner instance created
[java.lang.Enum.toString] [WARNING] Ignoring unsupported output: mp [ch.epfl.biop.atlas.aligner.MultiSlicePositioner]
[java.lang.Enum.toString] Exception in thread "AWT-EventQueue-0" [java.lang.Enum.toString] java.lang.NullPointerException[java.lang.Enum.toString] 
[java.lang.Enum.toString] 	at java.awt.Dimension.<init>(Dimension.java:111)[java.lang.Enum.toString] 
[java.lang.Enum.toString] 	at javax.swing.plaf.basic.BasicProgressBarUI.getPreferredSize(BasicProgressBarUI.java:825)[java.lang.Enum.toString] 
[java.lang.Enum.toString] 	at com.formdev.flatlaf.ui.FlatProgressBarUI.getPreferredSiz

nSlices = 97
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Found 97 validated image filenames.
97/97 [==============================] - 10s 108ms/step


## 2. Loading QuPath project

In [3]:
aligner.import_slices_from_qupath(slice_axis_initial_mm = 0.2,
                               increment_between_slices_mm = 0.08,
                               qupath_project= 'C:/Users/chiarutt/Desktop/abba-omero/abba-omero/project.qpproj') #'C:/Users/chiarutt/Desktop/ABBA_Workshop/PCB06-DemoProject/project.qpproj')

[java.lang.Enum.toString] The OMERO session for omero-server.epfl.ch needs to be initialized[java.lang.Enum.toString] 
[java.lang.Enum.toString] Session active : true
[java.lang.Enum.toString] [ERROR] Exception during event handling:
	[Event] org.scijava.display.event.DisplayActivatedEvent
	context = org.scijava.Context@76889e60
	consumed = false
	display = plugin:org.scijava.table.DefaultTableDisplay: type=interface org.scijava.table.Table, name=Omero - Connect, objects={[[success], [true]]}
	[Subscriber] Foreground
	[Method] protected void net.imagej.plugins.tools.AbstractColorTool.onEvent(org.scijava.display.event.DisplayActivatedEvent)
java.lang.AbstractMethodError: Method net/imagej/legacy/display/LegacyImageDisplayService.getActiveDatasetView()Lnet/imagej/display/DatasetView; is abstract
	at net.imagej.legacy.display.LegacyImageDisplayService.getActiveDatasetView(LegacyImageDisplayService.java)
	at net.imagej.plugins.tools.AbstractColorTool.drawIcon(AbstractColorTool.java:167)
	a

<java object 'org.scijava.command.CommandModule'>

In [4]:
print('nSlices = '+str(aligner.mp.getSlices().size()))
# ALL REGISTRATIONS AND COMMANDS BELOW ARE PERFORMED ON THE SELECTED SLICES!!
# since we want to register all of them, we select all of them
aligner.select_all_slices()

In [5]:
# we want to avoid saturation in the display. This does not matter for
# all registration methods EXCEPT for DeepSlice, which takes in rgb images
aligner.select_all_slices()
aligner.change_display_settings(0, 0, 800)
#abba.change_display_settings(1, 0, 1200) # fails because some slices have no channel 1

# programmatic way to show (or hide) sections and channels
aligner.get_bdv_view().setSelectedSlicesVisibility(True)
aligner.get_bdv_view().setSelectedSlicesVisibility(0, True)

## 4. DeepSlice Registration(s)

In [6]:
# REMOVE LABELS AND OVERVIEWS!!

# a first deepslice registration round : possible because it's the Allen CCF atlas, cut in coronal mode
# what's assumed : the sections are already in the correct order
aligner.register_slices_deepslice(channels=[0, 1])

<java object 'org.scijava.command.CommandModule'>

[java.lang.Enum.toString] Export of slice Slide_00.vsi [10x_18] done (18/97)[java.lang.Enum.toString] 
[java.lang.Enum.toString] Export of slice Slide_01.vsi [10x_10] done (32/97)[java.lang.Enum.toString] 
[java.lang.Enum.toString] Export of slice Slide_04.vsi [10x_06] done (70/97)[java.lang.Enum.toString] 
[java.lang.Enum.toString] Export of slice Slide_00.vsi [10x_16] done (16/97)[java.lang.Enum.toString] 
[java.lang.Enum.toString] Export of slice Slide_01.vsi [10x_09] done (31/97)[java.lang.Enum.toString] 
[java.lang.Enum.toString] Export of slice Slide_03.vsi [10x_12] done (64/97)[java.lang.Enum.toString] 
[java.lang.Enum.toString] Export of slice Slide_03.vsi [10x_10] done (62/97)[java.lang.Enum.toString] 
[java.lang.Enum.toString] Export of slice Slide_01.vsi [10x_11] done (33/97)[java.lang.Enum.toString] 
[java.lang.Enum.toString] Export of slice Slide_00.vsi [10x_08] done (8/97)[java.lang.Enum.toString] 
[java.lang.Enum.toString] Export of slice Slide_03.vsi [10x_04] done (56/9

In [ ]:
# second deepslice registration: because the slices are resampled for the registration,
# we usually get a slightly better positioning along z and cutting angle
# also: it's fast, and the combination of two affine transforms is
# an affine transform, so it's not like we are adding extra degrees of freedom
aligner.register_slices_deepslice(channels=[0, 1])

## 5. Elastix affine registration

In [ ]:
# a round of elastix registration, affine
# the channel 0 of the dataset (DAPI) is registered with the Nissl Channel of the atlas (0)
# and the channel 1 of the dataset (mainly autofluo) is registered with the autofluo channel of the atlas (1)
# these two channels have equal weights in the registration process
aligner.register_slices_elastix_affine(channels_slice_csv='0,1',
                                    channels_atlas_csv='0,1',
                                    pixel_size_micrometer=40)

## 6. Elastix spline registration

In [ ]:
# optional: a round of elastix registration, spline
# same channels as in the affine registration
# 5 control points along x = very coarse spline (and thus maybe unnecessary)
# abba.register_elastix_spline(
#    nb_control_points=5,
#    atlas_image_channels=[0, 1],
#    slice_image_channels=[0, 1],
#    pixel_size_micrometer=40).get()

# a round of elastix registration, affine
# same channels as in the affine registration 
# 16 control points = reasonable spline, which allows for local corrections, without deforming two much the section
aligner.register_slices_elastix_spline(channels_slice_csv='0,1',
                                    channels_atlas_csv='0,1',
                                    nb_control_points_x=16,
                                    pixel_size_micrometer=20)

## 7. Wait for end of all registrations

In [ ]:
# all tasks/registrations are enqueued and executed asynchronously
# if you need to wait before saving, then wait for all tasks to be finished:
aligner.wait_for_end_of_tasks()

In [ ]:
aligner.export_registration_to_qupath(erase_previous_file=True)

In [ ]:
# all tasks/registrations/exports are enqueued and executed asynchronously
# if you need to wait before saving, then wait for all tasks to be finished:
aligner.wait_for_end_of_tasks()

## 8. Saving the result

In [ ]:
save_dir = os.path.join(os.getcwd(), 'temp', 'notebook4', 'state')

if not os.path.exists(save_dir):
    os.makedirs(save_dir)


aligner.state_save(save_dir+"/state.abba") # full absolute path needed